In [ ]:
import sys
sys.path.append("..")
from src.decision_engine import DecisionEngine
from src.query_parser import Query_parser
from src.embeddings import EmbeddingsManager
import json

print("Initializing components...")

In [ ]:
# Initialize all components
parser=Query_parser()
em=EmbeddingsManager()
em.create_collection("policy_documents")
decision_engine=DecisionEngine()

print(f"✅ Using LLM: {decision_engine.provider} ({decision_engine.model})")


In [ ]:
print("Testing LLM connection...\n")
is_working = decision_engine.test_connection()

if is_working:
    print("\n✅ LLM is ready!")
else:
    print("\n❌ LLM connection failed. Check your setup.")

In [ ]:
print("="*80)
print("TEST 1: Simple Approved Case")
print("="*80 + "\n")

# Mock data
query_info = {
    'age': 46,
    'gender': 'male',
    'procedure': 'knee surgery',
    'location': 'Pune',
    'policy_duration_months': 3,
    'is_emergency': False
}

retrieved_docs = [
    "Knee surgery, hip replacement, and spinal procedures are covered at 80% of actual hospital bills. Coverage begins after 3 months of active policy for accident-related cases.",
    "Treatment in network hospitals in Pune, Mumbai, Bangalore, and Delhi are covered at 100% of scheduled amounts.",
    "Policyholders aged 18-50 are eligible for all surgical procedures without additional screening."
]

retrieved_metadata = [
    {'section': 'SURGICAL COVERAGE AND BENEFITS'},
    {'section': 'GEOGRAPHIC COVERAGE'},
    {'section': 'AGE ELIGIBILITY'}
]

decision = decision_engine.make_decision(query_info, retrieved_docs, retrieved_metadata)

print(f"Decision: {'✅ APPROVED' if decision.approved else '❌ REJECTED'}")
print(f"Amount: ₹{decision.amount:,}" if decision.amount else "Amount: Not specified")
print(f"Confidence: {decision.confidence}")
print(f"\nReasoning:")
print(f"  {decision.reasoning}")
print(f"\nRelevant Clauses:")
for clause in decision.relevant_clauses:
    print(f"  - {clause}")
if decision.risk_factors:
    print(f"\nRisk Factors:")
    for risk in decision.risk_factors:
        print(f"  ⚠️  {risk}")


In [ ]:
print("\n" + "="*80)
print("TEST 2: Rejection - Insufficient Policy Duration")
print("="*80 + "\n")

query_info_reject = {
    'age': 45,
    'gender': 'male',
    'procedure': 'knee surgery',
    'location': 'Mumbai',
    'policy_duration_months': 1,  # Too short!
    'is_emergency': False
}

decision2 = decision_engine.make_decision(query_info_reject, retrieved_docs, retrieved_metadata)

print(f"Decision: {'✅ APPROVED' if decision2.approved else '❌ REJECTED'}")
print(f"Confidence: {decision2.confidence}")
print(f"\nReasoning:")
print(f"  {decision2.reasoning}")
print(f"\nRisk Factors:")
for risk in decision2.risk_factors:
    print(f"  ⚠️  {risk}")

In [ ]:
print("\n" + "="*80)
print("TEST 3: Emergency Case - Immediate Coverage")
print("="*80 + "\n")

query_info_emergency = {
    'age': 28,
    'gender': 'male',
    'procedure': 'cardiac',
    'location': 'Delhi',
    'policy_duration_months': 1,
    'is_emergency': True  # Emergency!
}

emergency_docs = [
    "Emergency procedures covered immediately from policy start date with zero waiting period.",
    "Angioplasty, bypass surgery, and valve replacement covered at 90% of actual costs.",
    "Treatment in network hospitals in Pune, Mumbai, Bangalore, and Delhi are covered at 100% of scheduled amounts."
]

decision3 = decision_engine.make_decision(query_info_emergency, emergency_docs, retrieved_metadata)

print(f"Decision: {'✅ APPROVED' if decision3.approved else '❌ REJECTED'}")
print(f"Amount: ₹{decision3.amount:,}" if decision3.amount else "Amount: Not specified")
print(f"Confidence: {decision3.confidence}")
print(f"\nReasoning:")
print(f"  {decision3.reasoning}")


In [ ]:
decisions = [
    {"test": "approved_case", "decision": decision.dict()},
    {"test": "rejection_case", "decision": decision2.dict()},
    {"test": "emergency_case", "decision": decision3.dict()}
]

with open('../data/processed/sample_decisions.json', 'w') as f:
    json.dump(decisions, f, indent=2)

print("\n✅ Sample decisions saved to data/processed/sample_decisions.json")